In [34]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)
    

___

In [35]:
# builtin libraries
import logging
from typing import Union
from pathlib import Path
from collections import namedtuple

# secondary libraries
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader # load batches to the network
from fast_ctc_decode import beam_search, viterbi_search

# feito 
# from basecaller_tester import BasecallerTester as Tester
from feito.models import SimpleNet, Rodan
from feito.dataloaders.dataloader import DatasetONT
from feito.callbacks import CSVLogger, ModelCheckpoint
# ---- 
from collections import namedtuple

# types
_Path = Union[Path,str]

class BasecallerTester:
    
    def __init__(self, model, device, test_loader, path_fasta: _Path, rna: bool = True, use_viterbi = True):
        self.model=model.to(device) 
        self.device=device
        self.test_loader=test_loader # load signals
        self.path_fasta=path_fasta # save basecalled raw-reads
        self.alphabet="NACGU" if rna else "NACGT"
        self.search_algo=viterbi_search if use_viterbi  else beam_search

    def __call__(self,):
        print("Que me dice")
        # TODO: implement tester, considering accuracy

        # 1. generate output of a signal

        # 2. call viterbi or beam search to generate portion of a read
        
        # 3. send to a fasta file
        
        

    
    def predict_one_batch(self, batch):
        
        self.model.eval()
        X, y, output_len, target_len = (x.to(self.device) for x in batch)
        preds  = self.model(X)

    def signal_to_read(signal):

        pass

In [55]:
Args=namedtuple("Args", ["path_test", "batch_size", "model", "path_checkpoint", "device"])
args = Args(
"../data/subsample_val.hdf5",
16,
"Rodan",
"../output/training/checkpoints/Rodan-epoch4.pt",
"cpu",
)
    
PATH_TEST=args.path_test
BATCH_SIZE=args.batch_size
MODEL=args.model
DEVICE=args.device
PATH_CHECKPOINT=args.path_checkpoint

if DEVICE is None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else: 
    device = DEVICE
print("Device" , device)

model=eval(f"{MODEL}()")
model.to(device)
model.load_state_dict(torch.load(PATH_CHECKPOINT, map_location=torch.device('cpu')))
model_output_len = model.output_len

# dataset
dataset_test = DatasetONT(recfile=PATH_TEST, output_network_len=model_output_len)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)


Device cpu
CONFIG(vocab=['<PAD>', 'A', 'C', 'G', 'T'], activation='mish', sqex_activation='mish', dropout=0.1, sqex_reduction=32)
Activation Function is: mish
Activation Function is: mish


In [56]:
tester=BasecallerTester(
    model=model, 
    device=device,
    test_loader=dataloader_test,
    path_fasta="output/testing/basecalled_reads.fa"
)

# inference
tester()


Que me dice


In [60]:
batch = next(iter(dataloader_test))
model.eval()
X, y, output_len, target_len = (x.to(device) for x in batch)
preds = model(X)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [64]:
X.shape, preds.shape

(torch.Size([16, 1, 4096]), torch.Size([420, 16, 5]))

In [72]:
def signal_to_read(signal):
    return torch.Tensor([1,2])


axis=1
torch.stack([
    signal_to_read(signal) for signal in enumerate(torch.unbind(preds, dim=axis), 0)
], dim=axis)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])

In [73]:
preds

tensor([[[-0.0353, -4.3083, -5.3414, -5.8106, -4.3126],
         [-0.0493, -3.7497, -4.9249, -4.5319, -5.0215],
         [-0.0683, -4.5149, -3.8617, -4.1558, -3.9945],
         ...,
         [-0.1233, -3.0759, -3.8354, -4.7320, -3.2332],
         [-0.0968, -3.8178, -4.1701, -3.3622, -3.9044],
         [-0.0645, -4.5514, -3.7699, -5.2649, -3.7413]],

        [[-0.0260, -4.8810, -5.4244, -6.4000, -4.4186],
         [-0.0526, -3.5151, -5.2983, -4.4369, -5.3726],
         [-0.0571, -5.0049, -4.0832, -3.7914, -4.6691],
         ...,
         [-0.1561, -2.9753, -3.5557, -4.7091, -2.8841],
         [-0.0951, -3.4504, -4.4076, -3.4812, -4.1340],
         [-0.0738, -4.3499, -3.5965, -4.6835, -3.8352]],

        [[-0.0424, -4.6707, -4.8034, -6.2444, -3.8179],
         [-0.0763, -2.9426, -6.0223, -4.1446, -5.9987],
         [-0.1004, -4.4346, -3.5583, -3.1201, -4.5058],
         ...,
         [-0.2044, -2.7917, -3.1304, -4.6346, -2.6575],
         [-0.0886, -3.8203, -3.9195, -3.9144, -3.7713],
  

In [79]:
def signal_to_read(signal):
    return torch.Tensor([1,2])

np.apply_along_axis(signal_to_read, 1, preds.detach().numpy())

array([[[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]],

       [[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]],

       [[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]],

       ...,

       [[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]],

       [[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]],

       [[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.]]], dtype=float32)

In [ ]:
np.apply_along_axis()